# Orchestration and ML Pipelines

![Status](https://img.shields.io/static/v1.svg?label=Status&message=Ongoing&color=orange)

<!-- Place this tag where you want the button to render. -->
<a class="github-button" href="https://github.com/particle1331/steepest-ascent" data-color-scheme="no-preference: dark; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star particle1331/steepest-ascent on GitHub">Star</a>
<!-- Place this tag in your head or just before your close body tag. -->
<script async defer src="https://buttons.github.io/buttons.js"></script> 


In the previous module, we learned about experiment tracking and model registry.
In particular, we discussed how to get a candidate model and promote it from staging to production.
In this module, we learn how to automate this process, and having this scheduled with workflow orchestration &mdash; specifically, with [Prefect 2.0](https://orion-docs.prefect.io/).

Prefect allows us to programatically author, schedule, and monitor workflows. Prefect allows us to minimize time on **negative engineering**, i.e. coding against all possible causes of failure. This is a Sisyphean task as there are endless ways that elements of a data pipeline can fail. In practical terms, Prefect provides tools such as retries, concurrency, logging, a nice UI, tracking dependencies, a database, caching and serialization, parameterization of scheduled tasks, and more. As we shall see later, this adds observability to the whole data pipeline.

```{margin}
⚠️ **Attribution:** These are notes for [Module 3](https://github.com/DataTalksClub/mlops-zoomcamp/tree/main/03-orchestration) of the [MLOps Zoomcamp](https://github.com/DataTalksClub/mlops-zoomcamp). The MLOps Zoomcamp is a free course from [DataTalks.Club](https://github.com/DataTalksClub).
```

## Prefect basics

A **flow** in Prefect is simply a Python function. This consists of **tasks** which can be thought of as the atom of observability in Prefect. In practice, to create a flow, you simply convert functions that make it up into tasks. Consider [the following example](https://www.prefect.io/guide/blog/getting-started-prefect-2/#Makingourflowsbetterwithtasks) where we get data from an unreliable API, augment this data, then write the resulting data into a database. 

In [2]:
import random
from prefect import flow, task 


@task(retries=3)
def call_unreliable_api():
    choices = [{"data": 42}, "Failure"]
    res = random.choice(choices)
    if res == "Failure":
        raise Exception("Our unreliable service failed.")
    else:
        return res

@task
def augment_data(data: dict, msg: str):
    data["message"] = msg
    return data

@task
def write_to_database(data: dict):
    print(f"Wrote {data} to database successfully!")
    return "Success!"

@flow 
def pipeline(msg: str):
    api_result = call_unreliable_api()
    augmented_data = augment_data(data=api_result, msg=msg)
    write_to_database(augmented_data)


pipeline(0) # Augment data with zero.

23:05:51.772 | INFO    | prefect.engine - Created flow run 'ancient-sturgeon' for flow 'pipeline'
23:05:51.804 | INFO    | Flow run 'ancient-sturgeon' - Using task runner 'ConcurrentTaskRunner'
23:05:51.870 | WARNING | Flow run 'ancient-sturgeon' - No default storage is configured on the server. Results from this flow run will be stored in a temporary directory in its runtime environment.
23:05:51.967 | INFO    | Flow run 'ancient-sturgeon' - Created task run 'call_unreliable_api-48f93715-0' for task 'call_unreliable_api'
23:05:52.000 | INFO    | Flow run 'ancient-sturgeon' - Created task run 'augment_data-505b3e0c-0' for task 'augment_data'
23:05:52.010 | ERROR   | Task run 'call_unreliable_api-48f93715-0' - Encountered exception during execution:
Traceback (most recent call last):
  File "/Users/particle1331/miniforge3/envs/prefect/lib/python3.9/site-packages/prefect/engine.py", line 798, in orchestrate_task_run
    result = await run_sync_in_worker_thread(task.fn, *args, **kwargs)
 

Wrote {'data': 42, 'message': '0'} to database successfully!


Completed(message='All states completed.', type=COMPLETED, result=[Completed(message=None, type=COMPLETED, result={'data': 42, 'message': '0'}, task_run_id=abbf8182-dc8c-4e25-8169-baa5d141e288), Completed(message=None, type=COMPLETED, result={'data': 42, 'message': '0'}, task_run_id=fb891af2-e287-43bc-ae6e-3f6b2aed7a17), Completed(message=None, type=COMPLETED, result='Success!', task_run_id=eb4015c0-bed5-4ef9-b6b1-8f3ade94b2c5)], flow_run_id=53796cbf-187d-45c6-9a0d-c58b111e4a60)

Notice that this failed multiple times before pushing through. We can start the UI by calling `prefect orion start` in any directory (`.prefect` is saved in the system's root directory). This starts the Prefect Orion server in port 4200. Then, we can navigate around to find the `ancient-sturgeon` flow which as we have seen in the logs, was able to complete its execution.

```bash
❯ prefect orion start
Starting...

 ___ ___ ___ ___ ___ ___ _____    ___  ___ ___ ___  _  _
| _ \ _ \ __| __| __/ __|_   _|  / _ \| _ \_ _/ _ \| \| |
|  _/   / _|| _|| _| (__  | |   | (_) |   /| | (_) | .` |
|_| |_|_\___|_| |___\___| |_|    \___/|_|_\___\___/|_|\_|

Configure Prefect to communicate with the server with:

    prefect config set PREFECT_API_URL=http://127.0.0.1:4200/api

Check out the dashboard at http://127.0.0.1:4200



INFO:     Started server process [20557]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4200 (Press CTRL+C to quit)
```

Here we see that this flow started on `2022/06/10 11:05:51 PM` and ended on `2022/06/10 11:05:52 PM`. We also see the logs has the details of the exception when the API call failed. In the second tab, we can see the tasks that make up this flow. There is also the subflow tab which shows that we can call flows from a parent flow. One of the more interesting features of the UI is **Radar** on the right.

```{figure} ../../../img/hello-world-2.png
```

This shows the dependence between tasks. Notice the linear dependence, the `write_to_database` task depends on the `augment_data` task but not on the `call_unreliable_api` task. Having tasks arranged in concentric circles allow for a heirarchy of dependence. 

```{figure} ../../../img/hello-world-1.png
```

Finally, we have a flow which failed to complete all its tasks. Here all calls to the API failed despite the retries. We can clearly see from Radar where the flow has failed. This is super cool. Especially, when we have a dozens task and multiple subflows happening in our data pipeline.


```{figure} ../../../img/hello-world-3.png
```

<br>

**Remark.** Note also that geometrically there is more space available to grow the dependence tree due to nodes being farther apart as we move radially with a fixed angle, this also allows Radar to minimize edge crossing by combining radial and circumferential movement for the edges between task nodes. This is in comparison to traditional top-down / left-right approaches of drawing graphs. Furthermore, it turns out that Radar dynamically updates as tasks complete (or fails) its execution:

> Orion’s Radar is based on a structured, radial canvas upon which tasks are rendered as they are orchestrated. We developed a new layout algorithm that updates Radar as tasks run. It’s not just designed to handle simple workflows, but also those with massive dynamic fan-out, fan-in, sidecar tasks, and complex references. The algorithm optimizes readability through consistent node placement and minimal edge crossings. Users can zoom and pan across the canvas to discover and inspect tasks of interest. The mini-map, edge tracing, and node selection tools make workflow inspection a breeze. &mdash; [*Introducing Radar*](https://www.prefect.io/guide/blog/introducing-radar/) by Bill Palombi, Head of Product at Prefect



## MLflow runs as flow

In this section, we will write our code from the previous module for running modelling experiments as a flow in Prefect. Our idea is to have two flows: one for preprocessing the dataset such that the preprocessed datasets will be used by all experiment runs which will be the second flow.

```{margin}
[`utils.py`](https://github.com/particle1331/inefficient-networks/blob/57e38c5eb06ac3323035fb9f8d714870e397a39a/docs/notebooks/mlops/3-prefect/utils.py)
```
```python
@task
def load_training_dataframe(file_path, y_min=1, y_max=60):
    """Load data from disk and preprocess for training."""
    
    # Load data from disk
    data = pd.read_parquet(file_path)

    # Create target column and filter outliers
    data['duration'] = data.lpep_dropoff_datetime - data.lpep_pickup_datetime
    data['duration'] = data.duration.dt.total_seconds() / 60
    data = data[(data.duration >= y_min) & (data.duration <= y_max)]

    return data


@task
def fit_preprocessor(train_data):
    """Fit and save preprocessing pipeline."""

    # Unpack passed data
    y_train = train_data.duration.values
    X_train = train_data.drop('duration', axis=1)    

    # Initialize pipeline
    cat_features = ['PU_DO']
    num_features = ['trip_distance']

    preprocessor = make_pipeline(
        AddPickupDropoffPair(),
        SelectFeatures(cat_features + num_features),
        ConvertToString(cat_features),
        ConvertToDict(),
        DictVectorizer(),
    )

    # Fit only on train set
    preprocessor.fit(X_train, y_train)
    joblib.dump(preprocessor, artifacts / 'preprocessor.pkl')
    
    return preprocessor


@task
def create_model_features(preprocessor, train_data, valid_data):
    """Fit feature engineering pipeline. Transform training dataframes."""

    # Unpack passed data
    y_train = train_data.duration.values
    y_valid = valid_data.duration.values
    X_train = train_data.drop('duration', axis=1)
    X_valid = valid_data.drop('duration', axis=1)
    
    # Feature engineering
    X_train = preprocessor.transform(X_train)
    X_valid = preprocessor.transform(X_valid)

    return X_train, y_train, X_valid, y_valid


@flow
def preprocess_data(train_data_path, valid_data_path):
    """Preprocess data for model training."""

    train_data = load_training_dataframe(train_data_path)
    valid_data = load_training_dataframe(valid_data_path)
    
    preprocessor = fit_preprocessor(train_data)
    
    return create_model_features(preprocessor, train_data, valid_data).result()
```

Here we can see that the `preprocess_data` flow loads the datasets from disk, fits and saves a preprocessor, and then creates transformed features and targets for training the machine learning model. Note that we have to be careful here to make sure we don't use concurrent execution if using multiple since we may log different preprocessors. In this case, we only use on preprocessor for all experiments so this concern does not materialize.

Next, we will create a flow for executing experiment runs. Note that in the `main` flow we are passing around a [`PrefectFuture`](https://orion-docs.prefect.io/api-ref/prefect/futures/) object instead of Python objects. Futures represent the execution of a task and allow retrieval of the task run's state. This so that Prefect is able to track data dependency between tasks &mdash; converting to Python objects, e.g. using `.result()`, results in breaking this lineage. Note that once a future has been passed into the function, then we can treat this as a usual Python object. This is because the `task` decorator has done work to unpack the Future object into Python objects. For example, instead of defining:

```python
@task
def f(X, y):
    ...
```

We do:

```python
@task
def f(future):
    X, y = future
```

You will see notice this in the `xgboost_runs` and `lr_runs` tasks below. For the `main` flow, we execute the following sequentially:
setting up the connection to the experiment (not a task), subflow run for preprocessing the datasets for modelling, one run of the linear regression baseline model, and `num_xgb_runs` many runs of XGBoost hyperparameter optimization using the TPE algorithm. This ensures all resources are allocated to a single learning algorithm at each point in the flow run.

```{margin}
[`main.py`](https://github.com/particle1331/inefficient-networks/blob/fd937c097b9f59e171f263f0208b2407bb22efde/docs/notebooks/mlops/3-prefect/main.py)
```
```python
def objective(params, xgb_train, y_train, xgb_valid, y_valid):
    """Compute validation RMSE (one trial = one run)."""

    with mlflow.start_run():
        
        model = xgb.train(
            params=params,
            dtrain=xgb_train,
            num_boost_round=100,
            evals=[(xgb_valid, 'validation')],
            early_stopping_rounds=5
        )

        # MLflow logging
        ...

    return {'loss': rmse_valid, 'status': STATUS_OK}


@task
def xgboost_runs(num_runs, training_packet):
    """Run TPE algorithm on search space to minimize objective."""

    X_train, y_train, X_valid, y_valid = training_packet
    Xgb_train = xgb.DMatrix(X_train, label=y_train)
    Xgb_valid = xgb.DMatrix(X_valid, label=y_valid)


    search_space = {
        'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        'learning_rate': hp.loguniform('learning_rate', -3, 0),
        'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
        'objective': 'reg:squarederror',
        'seed': 42
    }

    best_result = fmin(
        fn=partial(
            objective, 
            xgb_train=Xgb_train, y_train=y_train, 
            xgb_valid=Xgb_valid, y_valid=y_valid,
        ),
        space=search_space,
        algo=tpe.suggest,
        max_evals=num_runs,
        trials=Trials()
    )


@task
def linreg_runs(training_packet):
    """Run linear regression training."""

    X_train, y_train, X_valid, y_valid = training_packet
    
    with mlflow.start_run():

        model = LinearRegression()
        model.fit(X_train, y_train)

        # MLflow logging
        ...

        
@flow(task_runner=SequentialTaskRunner())
def main(train_data_path, valid_data_path, num_xgb_runs=1):

    # Set and run experiment
    mlflow.set_tracking_uri("sqlite:///mlflow.db")
    mlflow.set_experiment("nyc-taxi-experiment")

    future = preprocess_data(train_data_path, valid_data_path)
    linreg_runs(future)
    xgboost_runs(num_xgb_runs, future)

```
